# Histopathologic Cancer Detection

# 1. Installing libraries, downloading the dataset

In [15]:
from numpy.random import seed
seed(101)

import pandas as pd
import numpy as np


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

import os
import cv2

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

We are not allowed to share the dataset. Download the dataset from Kaggle by pip installing kaggle, and running `kaggle competitions download -c histopathologic-cancer-detection` in your terminal. Then, extract files from the histopathologic-cancer-detection.zip folder into a new folder called 'data'.

In [16]:
IMAGE_SIZE = 96
IMAGE_CHANNELS = 3

SAMPLE_SIZE = 80000 # the number of images we use from each of the two classes

In [17]:
# See image count in each folder
print(len(os.listdir('data/train')))
print(len(os.listdir('data/test')))

220025
57458


# 2. Data Pre-processing

Now that the data has been properly loaded and set-up, we must now pre-process our data: in our case, we mainnly subset the data, augment the images, and split it into train and test.

Note: The data takes hours to download in its full size, leaving a high possibility of crashing the kernel, not to mention the time required to train the model. Thus, we slightly modified someone else's preprocessing instructions throughout this entire stage to get a smaller subset of the data. This will make it faster and easier for us to run and train our model.

https://www.kaggle.com/vbookshelf/cnn-how-to-use-160-000-images-without-crashing

## Removing anomalous images

In [18]:
# Create a Dataframe containing all images
data = pd.read_csv('data/train_labels.csv')

# Removing this image because it caused a training error previously
data[data['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']

# Removing this image because it's black
data[data['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']

print(data.shape)

(220025, 2)


## Augmenting image data

Justification for augmentation

This Kaggle challenge is a Machine Learning challenge. Machine learning however, requires plenty and diverse training data to accurately predict future data points without overfitting or underfitting. We have a lot of data points already, but also want to diversify them to prevent overfitting to the original data that may be too similar to each other already. We decided to augment the images in order to increase the diversity of images as inspired by this github repo below.

https://github.com/aleju/imgaug

In [19]:
#Function for augmenting data
from skimage.transform import rotate, AffineTransform
import cv2
from skimage.util import random_noise
import random
import os
from skimage import io
from skimage import img_as_ubyte

ORIGINAL_SIZE = IMAGE_SIZE      # original size of the images - do not change

# AUGMENTATION VARIABLES
CROP_SIZE = 90          # final size after crop
RANDOM_ROTATION = 3    # range (0-180), 180 allows all rotation variations, 0=no change
RANDOM_SHIFT = 2        # center crop shift in x and y axes, 0=no change. This cannot be more than (ORIGINAL_SIZE - CROP_SIZE)//2 
RANDOM_BRIGHTNESS = 7  # range (0-100), 0=no change
RANDOM_CONTRAST = 5    # range (0-100), 0=no change
RANDOM_90_DEG_TURN = 1  # 0 or 1= random turn to left or right

def readCroppedImage(path, augmentations = True):
    '''
    This is a custom function to convert an input image, augment it through
    random rotation, random x or y shift, random cropping, random flipping, 
    random changes in brightness and contrast, and returning it as an rgb tensor.
    '''
    # augmentations parameter is included for counting statistics from images, where we don't want augmentations
    
    # OpenCV reads the image in bgr format by default
    bgr_img = cv2.imread(path)
    # We flip it to rgb for visualization purposes
    b,g,r = cv2.split(bgr_img)
    rgb_img = cv2.merge([r,g,b])
    
    if(not augmentations):
        return rgb_img / 255
    
    #random rotation
    rotation = random.randint(-RANDOM_ROTATION,RANDOM_ROTATION)
    if(RANDOM_90_DEG_TURN == 1):
        rotation += random.randint(-1,1) * 90
    M = cv2.getRotationMatrix2D((48,48),rotation,1)   # the center point is the rotation anchor
    rgb_img = cv2.warpAffine(rgb_img,M,(96,96))
    
    #random x,y-shift
    x = random.randint(-RANDOM_SHIFT, RANDOM_SHIFT)
    y = random.randint(-RANDOM_SHIFT, RANDOM_SHIFT)
    
    # crop to center and normalize to 0-1 range
    start_crop = (ORIGINAL_SIZE - CROP_SIZE) // 2
    end_crop = start_crop + CROP_SIZE
    rgb_img = rgb_img[(start_crop + x):(end_crop + x), (start_crop + y):(end_crop + y)] / 255
    
    # Random flip
    flip_hor = bool(random.getrandbits(1))
    flip_ver = bool(random.getrandbits(1))
    if(flip_hor):
        rgb_img = rgb_img[:, ::-1]
    if(flip_ver):
        rgb_img = rgb_img[::-1, :]
        
    # Random brightness
    br = random.randint(-RANDOM_BRIGHTNESS, RANDOM_BRIGHTNESS) / 100.
    rgb_img = rgb_img + br
    
    # Random contrast
    cr = 1.0 + random.randint(-RANDOM_CONTRAST, RANDOM_CONTRAST) / 100.
    rgb_img = rgb_img * cr
    
    # clip values to 0-1 range
    rgb_img = np.clip(rgb_img, 0, 1.0)
    
    return img_as_ubyte(rgb_img)

In [20]:
#Augment test images randomly

images_path="data/train" #path to original images
augmented_path="data/train" # path to store augmented images
images=[] # to store paths of images from folder

for im in os.listdir(images_path):  # read image name from folder and append its path into "images" array     
    images.append(os.path.join(images_path,im))

images_to_generate=1 #you can change this value according to your requirement

for i in range(images_to_generate):    
    image=random.choice(images)
    id = image[11:-4]
    label = data[data['id'] == id].iloc[0]['label']
    data = data.append({"id":'augmented_'+id,'label':label},ignore_index=True)
    transformed_image= readCroppedImage(image)
    new_image_path= "train/augmented_%s.tif" %(id)
    cv2.imwrite(new_image_path, transformed_image) # save transformed image to path

#Save new label file which has the augmented images
data.to_csv('data/new_train_labels.csv')
data.shape

(220026, 2)

In [21]:
# Load the new csv that now includes the augmented images 
df_data = pd.read_csv('data/new_train_labels.csv')

# Check the class distribution
df_data['label'].value_counts()

0    130909
1     89117
Name: label, dtype: int64

## Balance the target distribution

As decided earlier with the variable SAMPLE_SIZE, we will subset our original data into 160000 images half labelled 0, the other labelled 1.